In [1]:
from hangar import Repository

In [2]:
repo = Repository(path = 'D:\Blog')

C:\Users\bintr\Anaconda3\envs\newconda\lib\site-packages\hangar-0.5.0.dev0-py3.7.egg\hangar\context.py:93: UserWarning: No repository exists at D:\Blog\.hangar, please use `repo.init()` method
  warnings.warn(msg, UserWarning)


In [3]:
repo.init(user_name='Name of User', user_email='some_email.com', remove_old=True)

Hangar Repo initialized at: D:\Blog\.hangar


'D:\\Blog\\.hangar'

In [4]:
co = repo.checkout(write=True)
co

Hangar WriterCheckout                
    Writer       : True                
    Base Branch  : master                
    Num Arraysets : 0                
    Num Metadata : 0


In [5]:
co.arraysets
co.metadata

Hangar Metadata                
    Writeable: True                
    Number of Keys: 0


In [6]:
import gzip
import pickle
import numpy as np
file_ = gzip.open('D:\Downloads\mnist.pkl.gz','rb')
train_DS, test_DS, validation_DS = pickle.load(file_, encoding = 'bytes')
# Extracting training images and labels into numpy arrays
image_arr = train_DS[0] # image_arr.shape:(50000, 784)
label_arr = train_DS[1] # label_arr.shape:(50000,)

In [7]:
# sample value in image_arr--> image_arr[0]-->(shape: (784,)
arr_set_img = co.arraysets.init_arrayset(name='train_image', prototype=np.array(image_arr[0]))
# sample value in label_arr--> label_arr[0]
arr_set_label = co.arraysets.init_arrayset(name='train_label', prototype=np.array(label_arr[0]))

In [8]:
co.arraysets

Hangar Arraysets                
    Writeable: True                
    Arrayset Names / Partial Remote References:                
      - train_image / False
      - train_label / False

In [9]:
arr_set_img

Hangar ArraysetDataWriter                 
    Arrayset Name             : train_image                
    Schema Hash              : 1=51076e42a840                
    Variable Shape           : False                
    (max) Shape              : (784,)                
    Datatype                 : <class 'numpy.float32'>                
    Named Samples            : True                
    Access Mode              : a                
    Number of Samples        : 0                
    Partial Remote Data Refs : False


In [10]:
arr_set_label

Hangar ArraysetDataWriter                 
    Arrayset Name             : train_label                
    Schema Hash              : 1=2a5b812f0f20                
    Variable Shape           : False                
    (max) Shape              : ()                
    Datatype                 : <class 'numpy.int64'>                
    Named Samples            : True                
    Access Mode              : a                
    Number of Samples        : 0                
    Partial Remote Data Refs : False


In [11]:
with arr_set_img, arr_set_label:
    for index, image in enumerate(image_arr):
        arr_set_img.add(data=image, name=index)
        arr_set_label.add(data=np.array(label_arr[index]), name=str(index))

In [14]:
co

Hangar WriterCheckout                
    Writer       : True                
    Base Branch  : master                
    Num Arraysets : 2                
    Num Metadata : 0


In [13]:
arr_set_img

Hangar ArraysetDataWriter                 
    Arrayset Name             : train_image                
    Schema Hash              : 1=51076e42a840                
    Variable Shape           : False                
    (max) Shape              : (784,)                
    Datatype                 : <class 'numpy.float32'>                
    Named Samples            : True                
    Access Mode              : a                
    Number of Samples        : 50000                
    Partial Remote Data Refs : False


In [ ]:
co.close()

In [17]:
repo.log() 
repo.list_branches()

['master']

In [18]:
first_commit_hash = co.commit('first commit')

In [19]:
repo.log()

* a=654fc9d4cd8cc0e841a535e129be030f26244a87 (master) : first commit


In [20]:
co.close()

In [22]:
branch_1 = repo.create_branch(name = 'branch_1') 
#the new branch points to our first commit by default.
branch_2 = repo.create_branch(name='branch_2', base_commit=first_commit_hash) 
#we can also specify the pointer to the first commit manually specifying it.

In [24]:
repo.log()

* a=654fc9d4cd8cc0e841a535e129be030f26244a87 (branch_1) (branch_2) (master) : first commit


In [29]:
repo.list_branches()

['branch_1', 'branch_2', 'master']

In [31]:
co = repo.checkout(write=True, branch='branch_2') #remember to mention the branch name when checking out
co.arraysets

Hangar Arraysets                
    Writeable: True                
    Arrayset Names / Partial Remote References:                
      - train_image / False
      - train_label / False

In [33]:
co.arraysets['train_image']

Hangar ArraysetDataWriter                 
    Arrayset Name             : train_image                
    Schema Hash              : 1=51076e42a840                
    Variable Shape           : False                
    (max) Shape              : (784,)                
    Datatype                 : <class 'numpy.float32'>                
    Named Samples            : True                
    Access Mode              : a                
    Number of Samples        : 50000                
    Partial Remote Data Refs : False


In [35]:
arr_ = co.arraysets['train_image'][0]

In [36]:
arr_ = co.arraysets['train_image'][0]
new_arr_ = arr_ + 10 #making a change in the arrayset sample 1
co['train_image', '1'] = new_arr_ #adding the changed the arrayset as the new arrayset sample(using a different method of add, you can also use the previous .add() method)
co.commit('commit on `branch_2` by adding a new sample')
co.close()

In [38]:
repo.log()

* a=ffdc5c384d1b875ea6d385d60531b269ec91f7d0 (branch_2) : commit on `branch_2` by adding a new sample
* a=654fc9d4cd8cc0e841a535e129be030f26244a87 (branch_1) (master) : first commit
